This notebook creates the following objects:
 - ```hardware_setup```: An object of type HardwareSetup and the central object
 - ```tawg```: An object that represents that physical TaborAWG
 - ```tchannelpair_AB``` and ```tchannelpair_CD```: Objects that represents a pair of channels each
 - ```alazar_DAC```: An object that represents the Alazar data acquisition card

Furthermore, the alazar will create a 10MHz reference clock that the AWG expects to receive at its ref clock input

In [1]:
from qupulse.hardware.setup import HardwareSetup, PlaybackChannel, MarkerChannel
hardware_setup = HardwareSetup()

In [15]:
from qupulse.hardware.awgs.tabor import TaborChannelPair, TaborAWGRepresentation
import pyvisa

def find_tabor_address():
    known_instruments = pyvisa.ResourceManager().list_resources()
    
    # find tabor AWG address
    tabor_address = None
    for address in known_instruments:
        if r'0x168C::0x2184' in address:
            tabor_address = address
            break
    if tabor_address is None:
        raise RuntimeError('Could not locate TaborAWG')
    
    return tabor_address

# small wrapper around pytabor
tawg = TaborAWGRepresentation(find_tabor_address(), reset=True)
tawg.paranoia_level = 2

# actual awg driver object
tchannelpair_AB = TaborChannelPair(tawg, (1, 2), 'TABOR_AB')
tchannelpair_CD = TaborChannelPair(tawg, (3, 4), 'TABOR_CD')

In [3]:
hardware_setup.set_channel('TABOR_A', PlaybackChannel(tchannelpair_AB, 0))
hardware_setup.set_channel('TABOR_B', PlaybackChannel(tchannelpair_AB, 1))
hardware_setup.set_channel('TABOR_A_MARKER', MarkerChannel(tchannelpair_AB, 0))
hardware_setup.set_channel('TABOR_B_MARKER', MarkerChannel(tchannelpair_AB, 1))

hardware_setup.set_channel('TABOR_C', PlaybackChannel(tchannelpair_CD, 0))
hardware_setup.set_channel('TABOR_D', PlaybackChannel(tchannelpair_CD, 1))
hardware_setup.set_channel('TABOR_C_MARKER', MarkerChannel(tchannelpair_CD, 0))
hardware_setup.set_channel('TABOR_D_MARKER', MarkerChannel(tchannelpair_CD, 1))

In [4]:
import atsaverage.atsaverage
import atsaverage.core

atsaverage.atsaverage.Logger.log_path = r'C:\Users\Public\AtsAverageLog'
atsaverage.atsaverage.Logger.rotate_file()

In [6]:
atsaverage.atsaverage.Logger.log_path = r'C:\Users\Public\AtsAverageLog'
atsaverage.atsaverage.Logger.rotate_file()
from qupulse.hardware.dacs.alazar import AlazarCard

alazar_DAC = AlazarCard(atsaverage.core.getLocalCard(1, 1))

def get_alazar_config():
    from atsaverage import alazar
    from atsaverage.config import ScanlineConfiguration, CaptureClockConfiguration, EngineTriggerConfiguration,\
        TRIGInputConfiguration, InputConfiguration
        
    r = 2.5
    rid = alazar.TriggerRangeID.etr_2V5

    trig_level = int((r + 0.15) / (2*r) * 255)
    assert 0 <= trig_level < 256

    config = ScanlineConfiguration()
    config.triggerInputConfiguration = TRIGInputConfiguration(triggerRange=rid)
    config.triggerConfiguration = EngineTriggerConfiguration(triggerOperation=alazar.TriggerOperation.J,
                                                             triggerEngine1=alazar.TriggerEngine.J,
                                                             triggerSource1=alazar.TriggerSource.external,
                                                             triggerSlope1=alazar.TriggerSlope.positive,
                                                             triggerLevel1=trig_level,
                                                             triggerEngine2=alazar.TriggerEngine.K,
                                                             triggerSource2=alazar.TriggerSource.disable,
                                                             triggerSlope2=alazar.TriggerSlope.positive,
                                                             triggerLevel2=trig_level)
    config.captureClockConfiguration = CaptureClockConfiguration(source=alazar.CaptureClockType.internal_clock,
                                                                 samplerate=alazar.SampleRateID.rate_100MSPS)
    config.inputConfiguration = 4*[InputConfiguration(input_range=alazar.InputRangeID.range_1_V)]
    config.totalRecordSize = 0
    config.aimedBufferSize = 10*config.aimedBufferSize

    # is set automatically
    assert config.totalRecordSize == 0
    
    return config
alazar_DAC.config = get_alazar_config()

hardware_setup.register_dac(alazar_DAC)

In [ ]:
# 10MHz clock out on AUX
atsaverage.alazar.ConfigureAuxIO(alazar_DAC.card.handle, atsaverage.alazar.AUX_IO_Mode.out_pacer, 10)

# 10MHz ref clock in
tawg.send_cmd(':ROSC:FREQ 10M')
tawg.send_cmd(':ROSC:SOUR EXT')